In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from scipy import stats
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df_csv = pd.read_csv("diabetes.csv")
df_csv

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
print("Creando la columna cualitativa 'Grupo_Edad'...")

# Definimos los límites (bins) para los grupos de edad:
bins = [0, 30, 50, np.inf]

# Definimos las etiquetas (nombres) para esos grupos
labels = ['Joven (0-30)', 'Adulto (31-50)', 'Mayor (51+)']

# Usamos la función pd.cut() para crear la nueva columna
# Asegúrate de que tu archivo tenga una columna llamada 'Age'
df_csv['Grupo_Edad'] = pd.cut(df_csv['Age'], bins=bins, labels=labels, right=True)

print("--- DataFrame Modificado (con la nueva columna 'Grupo_Edad') ---")
print(df_csv.head())

# Verificamos los tipos de datos (dtypes) para confirmar
print("\n--- Tipos de Datos (Dtypes) ---")
print(df_csv.dtypes)

Creando la columna cualitativa 'Grupo_Edad'...
--- DataFrame Modificado (con la nueva columna 'Grupo_Edad') ---
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome      Grupo_Edad  
0                     0.627   50        1  Adulto (31-50)  
1                     0.351   31        0  Adulto (31-50)  
2                     0.672   32        1  Adulto (31-50)  
3                     0.167   21        0    Joven (0-30)  
4                     2.288   33        1  Adulto (31-50)  

--- Tipos de Datos (Dtypes) ---
Pregnancies                    int64
Glucose                  

In [7]:
df_csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Grupo_Edad
0,6,148,72,35,0,33.6,0.627,50,1,Adulto (31-50)
1,1,85,66,29,0,26.6,0.351,31,0,Adulto (31-50)
2,8,183,64,0,0,23.3,0.672,32,1,Adulto (31-50)
3,1,89,66,23,94,28.1,0.167,21,0,Joven (0-30)
4,0,137,40,35,168,43.1,2.288,33,1,Adulto (31-50)
...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,Mayor (51+)
764,2,122,70,27,0,36.8,0.340,27,0,Joven (0-30)
765,5,121,72,23,112,26.2,0.245,30,0,Joven (0-30)
766,1,126,60,0,0,30.1,0.349,47,1,Adulto (31-50)


In [ ]:

# --- 1. Seleccionar SOLO las variables cuantitativas (X) ---
# Excluimos la variable objetivo 'Outcome' y la cualitativa 'Grupo_Edad'
quantitative_vars = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                       'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# Creamos un nuevo DataFrame solo con estas columnas
df_quant = df_csv[quantitative_vars]

print(f"Analizando {len(quantitative_vars)} variables cuantitativas...")

# --- 2. Realizar los cálculos del inciso 3 ---

# Usamos describe() para obtener la mayoría de las estadísticas
# .T transpone la tabla para que las variables queden como filas
desc = df_quant.describe().T

# Añadimos las columnas que faltan:

# Coeficiente de variación (CV = std / mean)
desc['coef_var'] = desc['std'] / desc['mean']

# Asimetría (skewness) - aplicado solo al DataFrame cuantitativo
desc['skewness'] = df_quant.apply(skew)

# Curtosis (kurtosis) - aplicado solo al DataFrame cuantitativo
desc['kurtosis'] = df_quant.apply(kurtosis)

# Reordenamos las columnas para que coincidan con la imagen (opcional pero prolijo)
column_order = ['mean', 'std', 'coef_var', 'min', '25%', '50%', '75%', 'max', 'skewness', 'kurtosis']
desc = desc[column_order]

# Renombramos percentiles para mayor claridad
desc = desc.rename(columns={
    '25%': 'Q1 (25%)',
    '50%': 'Q2 (median)',
    '75%': 'Q3 (75%)'
})

# --- 3. Mostrar la tabla final ---
print("\n--- Tabla de Análisis Descriptivo (Variables Cuantitativas) ---")
display(desc)

In [ ]:
for var in df_quant.columns:
    
    # Creamos una figura con 2 sub-gráficos (uno al lado del otro)
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    fig.suptitle(f'Análisis de la variable: {var}', fontsize=16, y=1.02)
    
    # Gráfico 1: Histograma y Densidad (agrupado por 'Outcome')
    # Usamos 'data=df' (el DataFrame COMPLETO) para poder usar 'Outcome' en 'hue'
    sns.histplot(data=df_csv, x=var, hue='Outcome', kde=True, ax=axes[0])
    axes[0].set_title('Histograma + Densidad (por Outcome)')
    
    # Gráfico 2: Boxplot (agrupado por 'Outcome')
    sns.boxplot(data=df_csv, x='Outcome', y=var, ax=axes[1])
    axes[1].set_title('Boxplot (por Outcome)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def detectar_outliers(df, var, eps=0.5, min_samples=5, mostrar_grafico=True):
    """
    Detecta outliers en una variable numérica de un DataFrame usando:
    - Método IQR
    - Método Z-score
    - DBSCAN
    
    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame con los datos
    var : str
        Nombre de la variable (columna) a analizar
    eps : float
        Parámetro epsilon para DBSCAN
    min_samples : int
        Número mínimo de muestras para DBSCAN
    mostrar_grafico : bool
        Si True, muestra el gráfico resumen de los outliers

    Retorna:
    --------
    dict con resultados estadísticos y booleanos de detección de outliers
    """

    subset = df[var].dropna().values  # eliminar posibles NaN

    # --- Paleta de colores ---
    palette = {
        "normal": "#1f77b4",       # azul
        "zscore": "#34a02c",       # verde
        "dbscan": "#d62728",       # rojo
    }

    # --- Cálculo de outliers ---
    Q1, Q3 = np.percentile(subset, [25, 75])
    IQR = Q3 - Q1
    lim_inf, lim_sup = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    out_iqr = (subset < lim_inf) | (subset > lim_sup)

    out_z = np.abs(zscore(subset)) > 3

    db = DBSCAN(eps=eps, min_samples=min_samples)
    out_db = db.fit_predict(subset.reshape(-1, 1)) == -1

    # --- Combinación de métodos ---
    out_total = out_iqr.astype(int) + out_z.astype(int) + out_db.astype(int)
    normales = out_total == 0
    solo_z = out_z & (out_total == 1)
    solo_db = out_db & (out_total == 1)


    # --- Reporte ---
    print(f"📊 Variable: {var}")
    print(f" - Outliers Z-score: {np.sum(out_z)}")
    print(f" - Outliers DBSCAN: {np.sum(out_db)}")
    print(f" - Total de datos: {len(subset)}")

    # --- Gráfico ---
    if mostrar_grafico:
        plt.figure(figsize=(12, 6))
        plt.title(f"Detección de outliers - {var}", fontsize=14, fontweight="bold")
        plt.xlabel(var, fontsize=12)
        sns.boxplot(x=subset, color="#f0f0f0")

        jitter = np.random.uniform(-0.05, 0.05, size=len(subset))
        plt.scatter(subset[normales], jitter[normales], color=palette["normal"], s=40, alpha=0.6, label="Normales")
        plt.scatter(subset[solo_z], jitter[solo_z], color=palette["zscore"], s=60, alpha=0.8, label="Z-score")
        plt.scatter(subset[solo_db], jitter[solo_db], color=palette["dbscan"], s=60, alpha=0.8, label="DBSCAN")

        # Líneas de referencia
        plt.axvline(Q1, color="#2A9D8F", linestyle="--", linewidth=1.2)
        plt.axvline(Q3, color="#2A9D8F", linestyle="--", linewidth=1.2)
        plt.axvline(lim_inf, color="#F4A261", linestyle=":", linewidth=1.2)
        plt.axvline(lim_sup, color="#F4A261", linestyle=":", linewidth=1.2)

        plt.yticks([])
        plt.legend(frameon=True)
        plt.tight_layout()
        plt.show()

    # --- Resultados en diccionario ---
    resultados = {
        "variable": var,
        "lim_inf": lim_inf,
        "lim_sup": lim_sup,
        "outliers_iqr": out_iqr,
        "outliers_zscore": out_z,
        "outliers_dbscan": out_db,
        "total_outliers": np.sum(out_total > 0)
    }

    return resultados

In [ ]:
detectar_outliers(df_csv, var="Pregnancies")

In [ ]:
detectar_outliers(df_csv, var="Glucose")

In [ ]:
detectar_outliers(df_csv, var="BloodPressure")

In [ ]:
detectar_outliers(df_csv, var="SkinThickness")

In [ ]:
detectar_outliers(df_csv, var="Insulin")

In [ ]:
detectar_outliers(df_csv, var="BMI")

In [ ]:
detectar_outliers(df_csv, var="DiabetesPedigreeFunction")

In [ ]:
detectar_outliers(df_csv, var="Age")

In [ ]:
def graficar_boxplot(df, columna, color="#A7C7E7"):
    """
    Grafica un boxplot (diagrama de caja) usando Plotly para una variable numérica.

    Parámetros:
    - df: DataFrame con los datos
    - columna: Nombre de la columna numérica
    - color: Color de la caja
    """
    print(f"\n📦 Boxplot de: {columna}")
    print("Este gráfico muestra la mediana, los cuartiles y posibles valores atípicos.")

    fig = go.Figure()
    fig.add_trace(go.Box(
        y=df[columna],
        name=columna,
        boxpoints='outliers',  # Muestra puntos atípicos
        marker_color=color,
        line=dict(color="black"),
        fillcolor=color,
        opacity=0.7
    ))

    fig.update_layout(
        title=f"Boxplot de {columna}",
        yaxis_title=columna,
        template="plotly_white",
        width=700,
        height=500
    )

    fig.show()

In [ ]:
def graficar_histograma(df, columna, color="#A7C7E7"):
    """
    Grafica un histograma simple para una variable numérica.

    Parámetros:
    - df: DataFrame con los datos.
    - columna: Nombre de la columna numérica a graficar.
    - color: Color del histograma (por defecto azul claro).
    """
    # Calcular número de bins usando regla de Freedman-Diaconis
    datos = df[columna].dropna()
    q75, q25 = np.percentile(datos, [75 ,25])
    iqr = q75 - q25
    bin_width = 2 * iqr / np.cbrt(len(datos))
    num_bins = int((datos.max() - datos.min()) / bin_width) if bin_width > 0 else 20
    if num_bins > 10:
        num_bins=num_bins
    else:
        num_bins= 15

    print(f"Variable: {columna}")
    print(f"Rango intercuartílico (IQR): {iqr:.2f}")
    print(f"Número de bins calculado: {num_bins}")

    # Obtener histograma como datos
    frecuencias, bordes = np.histogram(datos, bins=num_bins)

    # Imprimir intervalos y frecuencias
    print(f"\n📊 Histograma de: {columna}")
    print(f"IQR = {iqr:.2f} | Ancho de bin = {bin_width:.2f} | Bins = {num_bins}\n")
    print("Intervalo\t\tFrecuencia")
    for i in range(len(frecuencias)):
        print(f"[{bordes[i]:.2f}, {bordes[i+1]:.2f})\t\t{frecuencias[i]}")

    # Graficar histograma
    plt.figure(figsize=(10, 5))
    sns.histplot(datos, bins=num_bins, color=color, kde=True)
    plt.title(f"Histograma de {columna}")
    plt.xlabel(columna)
    plt.ylabel("Frecuencia")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
df_logs = df_csv.copy()

In [ ]:
df_logs['log_Pregnancies'] = np.log(df_csv['Pregnancies'] + 1)
print("Variable Fare sin transformación:")
graficar_histograma(df_csv, "Pregnancies")
graficar_boxplot(df_csv, "Pregnancies")  # Boxplot de la variable original

print("\nVaraible Fare con transformación Logarítmica:")
graficar_histograma(df_logs, "log_Pregnancies")
graficar_boxplot(df_logs, "log_Pregnancies")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_Glucose'] = np.log(df_csv['Glucose'] + 1)
print("Variable Glucose sin transformación:")
graficar_histograma(df_csv, "Glucose")
graficar_boxplot(df_csv, "Glucose")  # Boxplot de la variable original

print("\nVaraible Glucose con transformación Logarítmica:")
graficar_histograma(df_logs, "log_Glucose")
graficar_boxplot(df_logs, "log_Glucose")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_BloodPressure'] = np.log(df_csv['BloodPressure'] + 1)
print("Variable BloodPressure sin transformación:")
graficar_histograma(df_csv, "BloodPressure")
graficar_boxplot(df_csv, "BloodPressure")  # Boxplot de la variable original

print("\nVaraible BloodPressure con transformación Logarítmica:")
graficar_histograma(df_logs, "log_BloodPressure")
graficar_boxplot(df_logs, "log_BloodPressure")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_SkinThickness'] = np.log(df_csv['SkinThickness'] + 1)
print("Variable SkinThickness sin transformación:")
graficar_histograma(df_csv, "SkinThickness")
graficar_boxplot(df_csv, "SkinThickness")  # Boxplot de la variable original

print("\nVaraible SkinThickness con transformación Logarítmica:")
graficar_histograma(df_logs, "log_SkinThickness")
graficar_boxplot(df_logs, "log_SkinThickness")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_Insulin'] = np.log(df_csv['Insulin'] + 1)
print("Variable Insulin sin transformación:")
graficar_histograma(df_csv, "Insulin")
graficar_boxplot(df_csv, "Insulin")  # Boxplot de la variable original

print("\nVaraible Insulin con transformación Logarítmica:")
graficar_histograma(df_logs, "log_Insulin")
graficar_boxplot(df_logs, "log_Insulin")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_BMI'] = np.log(df_csv['BMI'] + 1)
print("Variable BMI sin transformación:")
graficar_histograma(df_csv, "BMI")
graficar_boxplot(df_csv, "BMI")  # Boxplot de la variable original

print("\nVaraible BMI con transformación Logarítmica:")
graficar_histograma(df_logs, "log_BMI")
graficar_boxplot(df_logs, "log_BMI")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_DiabetesPedigreeFunction'] = np.log(df_csv['DiabetesPedigreeFunction'] + 1)
print("Variable DiabetesPedigreeFunction sin transformación:")
graficar_histograma(df_csv, "DiabetesPedigreeFunction")
graficar_boxplot(df_csv, "DiabetesPedigreeFunction")  # Boxplot de la variable original

print("\nVaraible DiabetesPedigreeFunction con transformación Logarítmica:")
graficar_histograma(df_logs, "DiabetesPedigreeFunction")
graficar_boxplot(df_logs, "DiabetesPedigreeFunction")  # Boxplot de la variable transformada 


In [ ]:
df_logs['log_Age'] = np.log(df_csv['Age'] + 1)
print("Variable Age sin transformación:")
graficar_histograma(df_csv, "Age")
graficar_boxplot(df_csv, "Age")  # Boxplot de la variable original

print("\nVaraible Age con transformación Logarítmica:")
graficar_histograma(df_logs, "log_Age")
graficar_boxplot(df_logs, "log_Age")  # Boxplot de la variable transformada 


In [ ]:
df_box = df_csv.copy()

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['Pregnancies'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_Pregnancies'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable Pregnancies sin transformación:")
graficar_histograma(df_csv, "Pregnancies")
graficar_boxplot(df_csv, "Pregnancies")

print("\nVariable Pregnancies con transformación Box-Cox:")
graficar_histograma(df_box, "box_Pregnancies")
graficar_boxplot(df_box, "box_Pregnancies")



In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['Glucose'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_Glucose'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable Glucose sin transformación:")
graficar_histograma(df_csv, "Glucose")
graficar_boxplot(df_csv, "Glucose")

print("\nVariable Glucose con transformación Box-Cox:")
graficar_histograma(df_box, "box_Glucose")
graficar_boxplot(df_box, "box_Glucose")

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['BloodPressure'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_BloodPressure'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable BloodPressure sin transformación:")
graficar_histograma(df_csv, "BloodPressure")
graficar_boxplot(df_csv, "BloodPressure")

print("\nVariable BloodPressure con transformación Box-Cox:")
graficar_histograma(df_box, "box_BloodPressure")
graficar_boxplot(df_box, "box_BloodPressure")

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['SkinThickness'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_SkinThickness'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable SkinThickness sin transformación:")
graficar_histograma(df_csv, "SkinThickness")
graficar_boxplot(df_csv, "SkinThickness")

print("\nVariable SkinThickness con transformación Box-Cox:")
graficar_histograma(df_box, "box_SkinThickness")
graficar_boxplot(df_box, "box_SkinThickness")

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['Insulin'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_Insulin'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable Insulin sin transformación:")
graficar_histograma(df_csv, "Insulin")
graficar_boxplot(df_csv, "Insulin")

print("\nVariable Insulin con transformación Box-Cox:")
graficar_histograma(df_box, "box_Insulin")
graficar_boxplot(df_box, "box_Insulin")

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['BMI'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_BMI'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable BMI sin transformación:")
graficar_histograma(df_csv, "BMI")
graficar_boxplot(df_csv, "BMI")

print("\nVariable BMI con transformación Box-Cox:")
graficar_histograma(df_box, "box_BMI")
graficar_boxplot(df_box, "box_BMI")

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['DiabetesPedigreeFunction'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_DiabetesPedigreeFunction'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable DiabetesPedigreeFunction sin transformación:")
graficar_histograma(df_csv, "DiabetesPedigreeFunction")
graficar_boxplot(df_csv, "DiabetesPedigreeFunction")

print("\nVariable DiabetesPedigreeFunction con transformación Box-Cox:")
graficar_histograma(df_box, "box_DiabetesPedigreeFunction")
graficar_boxplot(df_box, "box_DiabetesPedigreeFunction")

In [ ]:
# 1. Preparamos los datos (¡AÑADIMOS EL +1!)
data_para_transformar = df_csv['Age'] + 1

# 2. Ejecutamos Box-Cox y capturamos AMBAS salidas
#    - 'datos_transformados' (el array de 50 filas)
#    - 'lambda_optimo' (el número único)
datos_transformados, lambda_optimo = stats.boxcox(data_para_transformar)

# 3. Asignamos SOLAMENTE el array de datos transformados a la columna
df_box['box_Age'] = datos_transformados

# (Opcional) Imprimimos el lambda para saber qué hizo
print(f"El lambda óptimo encontrado fue: {lambda_optimo}")

# --- Ahora tu código de gráficas funcionará ---
print("\nVariable Age sin transformación:")
graficar_histograma(df_csv, "Age")
graficar_boxplot(df_csv, "Age")

print("\nVariable Age con transformación Box-Cox:")
graficar_histograma(df_box, "box_Age")
graficar_boxplot(df_box, "box_Age")

In [ ]:
# Solución Rápida:
skew_transformado = df_box.skew(numeric_only=True)

print("--- Asimetría (Skewness) del DataFrame Transformado (solo numéricos) ---")
display(skew_transformado)

In [ ]:
# Solución Rápida:
skew_transformado = df_logs.skew(numeric_only=True)

print("--- Asimetría (Skewness) del DataFrame Transformado (solo numéricos) ---")
display(skew_transformado)

In [ ]:
# --- 1. Análisis de la Variable Cualitativa (X: Grupo_Edad) ---
#    (Instrucción: Diagramas de barras agrupados por Y y tabla de contingencia)

print("--- 1. Análisis de Variable Cualitativa (Grupo_Edad) ---")

# A. Gráfico de barras agrupado por la variable objetivo (Outcome)
plt.figure(figsize=(10, 6))
sns.countplot(data=df_csv, x='Grupo_Edad', hue='Outcome', palette='pastel')
plt.title('Distribución de Grupo_Edad agrupado por Outcome (Diabetes)', fontsize=16)
plt.xlabel('Grupo de Edad')
plt.ylabel('Conteo')
plt.legend(title='Outcome', labels=['No Diabetes (0)', 'Diabetes (1)'])
plt.show()

# B. Tabla de Contingencia (Distribución conjunta)
print("\n--- Tabla de Contingencia (Grupo_Edad vs Outcome) ---")
# pd.crosstab() es la función exacta para esto
contingency_table = pd.crosstab(df_csv['Grupo_Edad'], df_csv['Outcome'])

# Renombramos columnas para claridad
contingency_table = contingency_table.rename(columns={0: 'No Diabetes (0)', 1: 'Diabetes (1)'})
display(contingency_table)

# Opcional: Ver la misma tabla pero con porcentajes (distribución por fila)
print("\n--- Tabla de Contingencia (Porcentajes por Grupo de Edad) ---")
contingency_pct = pd.crosstab(df_csv['Grupo_Edad'], df_csv['Outcome'], normalize='index')
contingency_pct = contingency_pct.rename(columns={0: 'No Diabetes (0)', 1: 'Diabetes (1)'})
# Usamos .applymap para formatear los porcentajes de forma legible
display(contingency_pct.applymap(lambda x: f"{x*100:.1f}%"))


# --- 2. Análisis de la Variable Objetivo (Y: Outcome) ---
#    (Instrucción: Analizar balance, gráfico de torta y gráfico de barras)

print("\n\n--- 2. Análisis de Balance de Clases (Variable Objetivo: Outcome) ---")

# A. Analizar el balance de clases (conteo y porcentaje)
class_counts = df_csv['Outcome'].value_counts()
class_pct = df_csv['Outcome'].value_counts(normalize=True) * 100

print("\nConteo de Clases (Outcome):")
display(class_counts)
print("\nPorcentaje de Clases (Outcome):")
display(class_pct)

# Comentario de análisis:
print(f"Análisis: El dataset está desbalanceado. La clase 0 (No Diabetes) representa el {class_pct[0]:.1f}% "
      f"mientras que la clase 1 (Diabetes) es el {class_pct[1]:.1f}%.")


# B y C. Gráfico de Torta y Gráfico de Barras (para la variable objetivo)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Análisis de Balance de Clases para la Variable Objetivo (Outcome)', fontsize=16)

# Gráfico de Torta (Pie Chart)
axes[0].pie(class_counts, labels=[f'No Diabetes (0)', f'Diabetes (1)'], 
            autopct='%1.1f%%', startangle=90, colors=['#99d594', '#fc8d59'])
axes[0].set_title('Gráfico de Torta (Balance de Clases)')
axes[0].axis('equal') # Asegura que el gráfico sea circular

# Gráfico de Barras
sns.countplot(data=df_csv, x='Outcome', ax=axes[1], palette=['#99d594', '#fc8d59'])
axes[1].set_title('Gráfico de Barras (Conteo de Clases)')
axes[1].set_xlabel('Outcome')
axes[1].set_ylabel('Conteo Total')
# Seteamos las etiquetas del eje X para que sean claras
axes[1].set_xticklabels([f'No Diabetes (0)\n{class_counts[0]} obs.', f'Diabetes (1)\n{class_counts[1]} obs.'])

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajusta el layout para el supertítulo
plt.show()